In [46]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import numpy as np
import random
import matplotlib.pyplot as plt
import os

try:
    os.chdir(os.path.join(os.getcwd(), '../../catchgame/'))
except:
    print("already in directory")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device} is available")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
already in directory
cuda is available


In [47]:
from modules.dqn.network import QNetworkConv
from world import CatchEnv

n_actions = 3
n_episodes = 1000
batch_size = 32
gamma = 0.99
eps_start = 1.0
eps_end = 0.1
eps_decay = 100000
target_update = 1000

CATCH_ENV_NUM_STATES = 4
CATCH_ENV_WORLD_SIZE = (84, 84)
CATCH_ENV_POSSIBLE_ACTIONS = [0, 1, 2]

spatial_size = CATCH_ENV_WORLD_SIZE
qnet = QNetworkConv(
    in_channels=CATCH_ENV_NUM_STATES, 
    spatial_size=spatial_size, 
    output_size=len(CATCH_ENV_POSSIBLE_ACTIONS)
).to(device)
qnet(torch.randn(batch_size, CATCH_ENV_NUM_STATES, *spatial_size, device=device)).shape


torch.Size([32, 3])

In [40]:
env = CatchEnv()
env.reset_random()
next_state, reward, done, = env.step(action=1)

next_state.shape, reward, done

((84, 84, 4), 0, False)